###Import Library###

In [1]:
import nltk
import json
import pandas as pd

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

###Load the json data###

In [4]:
with open('pavbhaji.json', 'r') as file:
    data = json.load(file)

In [5]:
# data1=pd.read_json("pavbhaji.json")
# data1.to_csv("op.csv",index=False)


In [6]:
data[1]

{'dimensions': {'height': 750, 'width': 750},
 'display_url': 'https://instagram.fpnq3-1.fna.fbcdn.net/vp/b0d33fc5b1b7d5e41759678f471aff19/5B8705CA/t51.2885-15/e15/36848355_187489742110497_7922878806031859712_n.jpg',
 'edge_liked_by': {'count': 12041},
 'edge_media_preview_like': {'count': 12041},
 'edge_media_to_caption': {'edges': [{'node': {'text': 'देसी स्टाइल पाव भाजी/Desi Style Paav Bhaji 🇮🇳 ❤️ 🏠 ❤️🏠❤️🏠❤️🏠❤️\n_______\nGood Afternoon Instafamily ❤️\n_______\nक्या आप को पाव भाजी पसन्द हैं?😍\nलाइक एंड कॉमेंट करके बतायें😋😍\n_______\nIngredients:\n_______\n8-10 Potatoes\n1 Cauliflower \n2-3 Carrot\n1 Cup Peas\n3 Cups Water\n4 tbsp oil\n1 tbsp Ginger Garlic Paste\n2 Onions - Finely Chopped \n3 Tomatoes - Finaly Chopped \n1 tbsp Red Chilli Powder \n1 tbsp Kashmiri Red Chilli\n2-3 tbsp Pav Bhaji Masala\nSalt to Taste\nCoriander to Garnish\n_______\nRecipe:\n_______\n- Pressure cook roughly chopped potatoes, cauliflower, carrot & peas with 3 cups of water to 3 whistles. - Mash the boiled 

#### Extract texts and labels###

In [50]:
texts = []
labels = []
#pattern =re.compile(r'\bbhaji\b', re.IGNORECASE)
for entry in data:
    try:
        text = entry["edge_media_to_caption"]["edges"][0]["node"]["text"]
        label = 1 if "#pavbhaji" in text.lower()  or "pav bhaji" in text.lower() else 0   #pattern.search(text)
        texts.append(text)
        labels.append(label)
    except (KeyError, IndexError) as e:
        print(f"Skipping entry due to error: {e}")
        print(entry)

Skipping entry due to error: list index out of range
{'comments_disabled': False, 'dimensions': {'height': 1080, 'width': 1080}, 'display_url': 'https://instagram.fpnq3-1.fna.fbcdn.net/vp/c6dac55b57c2e87ecb96315519b1459c/5C02F331/t51.2885-15/e35/38885028_254267458551198_8280479196141060096_n.jpg', 'edge_liked_by': {'count': 17}, 'edge_media_preview_like': {'count': 17}, 'edge_media_to_caption': {'edges': []}, 'edge_media_to_comment': {'count': 2}, 'id': '1850713497443087014', 'is_video': False, 'location': {'has_public_page': True, 'id': '216978098', 'name': 'Mumbai, Maharashtra', 'slug': 'mumbai-maharashtra'}, 'owner': {'id': '8430771130'}, 'shortcode': 'BmvDh7UHbKm', 'taken_at_timestamp': 1534842262, 'thumbnail_resources': [{'config_height': 150, 'config_width': 150, 'src': 'https://instagram.fpnq3-1.fna.fbcdn.net/vp/84b28b95442e3eedaa87eed3dde75f26/5C314973/t51.2885-15/e35/s150x150/38885028_254267458551198_8280479196141060096_n.jpg'}, {'config_height': 240, 'config_width': 240, 'src

In [51]:
# Create a DataFrame
df = pd.DataFrame({'Text': texts, 'Label': labels})

In [52]:
df[1:5]


,Text,Label
1,देसी स्टाइल पाव भाजी/Desi Style Paav Bhaji 🇮🇳 ...,1
2,Pav Bhaji😍😍😋\n.\nFollow @street_food_chandigar...,1
3,Asli Makhan Pav bhaji 😍😋\nTag someone who love...,1
4,Tag a Pav bhaji lover 👇🏻 😋😋,1


In [53]:
df['Label'].value_counts() #data is imbalace

1    1490
0       9
Name: Label, dtype: int64

In [54]:
len(texts)

1499

###Text Preprocessing ###

In [55]:
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z]', ' ', text)
    words = word_tokenize(text)
    words = [word for word in words if word not in stopwords.words('english')]
    return ' '.join(words)

cleaned_text_features = [preprocess_text(text) for text in texts]

In [56]:
cleaned_text_features[1:5]

['desi style paav bhaji good afternoon instafamily ingredients potatoes cauliflower carrot cup peas cups water tbsp oil tbsp ginger garlic paste onions finely chopped tomatoes finaly chopped tbsp red chilli powder tbsp kashmiri red chilli tbsp pav bhaji masala salt taste coriander garnish recipe pressure cook roughly chopped potatoes cauliflower carrot peas cups water whistles mash boiled veggies keep aside heat oil wok add ginger garlic paste cook till aroma fades away add onions cook till turn light brown add tomatoes cook till corner leaves oil add red chilli powder kashmiri chilli powder pav bhaji masala mix add mashed veggies salt mix cook mints adjust consistency adding water stir mash veggies garnish coriander serve hot pav follow sofaridabad homemade food updates please turn post notification use sofaridabad get featured page pavbhaji streetstyle homemadewithlove homecooked cooking streetfood mumbaifoodie views eat tastyfood favourite dish carrot vegetarian vegetables healthyli

###Creating Model###

In [57]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dropout
from tensorflow.keras.regularizers import l2

###Generating onehot Represenation, Embedding layer, and Padding###

In [58]:
voc_size =5000

In [59]:
#onehot_rep
onehot_rep = [one_hot(words, voc_size)for words in cleaned_text_features ]

In [60]:
onehot_rep[0]

[2663,
 2343,
 731,
 4930,
 1061,
 4996,
 3675,
 4025,
 2298,
 3842,
 1058,
 3288,
 2663,
 1294,
 747,
 3842,
 3460,
 87,
 1034,
 1810,
 1527,
 4044,
 2063,
 427,
 3771,
 3626,
 3164,
 2412,
 549,
 1012,
 2791,
 4869,
 2283,
 2255,
 2206,
 1945,
 1034,
 33,
 4493,
 869,
 562,
 2347,
 2565,
 32,
 3678]

In [61]:
#padding
sent_length = 150

In [62]:
x_padded = pad_sequences(onehot_rep, maxlen=sent_length, padding='pre')
print(x_padded)

[[   0    0    0 ... 2565   32 3678]
 [4517 3934 4548 ... 4643 4631  408]
 [   0    0    0 ... 1012 1788 1078]
 ...
 [   0    0    0 ... 3614 2973 1785]
 [   0    0    0 ... 2603 4894 2116]
 [   0    0    0 ... 2725 4662 3243]]


###Generate Model (Embedding Layer, Dropoutlayer, LSTM with L2 Regularization, Dese layer)###

In [63]:
#creating model
embedding_vector_features = 100
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
#model.add(SpatialDropout1D(0.3))
model.add(LSTM(100, kernel_regularizer=l2(0.01),recurrent_regularizer=l2(0.01)))
model.add(Dropout(0.5))
model.add(Dense(1, activation= 'sigmoid'))


###Compile the model###

In [64]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [65]:
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 150, 100)          500000    
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 580501 (2.21 MB)
Trainable params: 580501 (2.21 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [66]:
print(model.predict(x_padded))

47/47 [==============================] - 2s 44ms/step
[[0.4995479 ]
 [0.50072306]
 [0.49830073]
 ...
 [0.5021452 ]
 [0.5027517 ]
 [0.5010066 ]]


In [67]:
###convert into array###

In [68]:
import numpy as np

In [69]:
x_final = np.array(x_padded)

In [70]:
y_final = np.array(labels)

###Split the train test data###

In [71]:
X_train, X_test, y_train, y_test = train_test_split(x_final, y_final, test_size=0.2, random_state=42)

###Apply SMOTE to balance the data###

In [72]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [73]:
(y_train_resampled).shape

(2386,)

In [74]:
counts = np.bincount(y_train_resampled)
counts
count_0 = counts[0]
count_1 = counts[1]



In [75]:
count_0

1193

In [76]:
y_test.shape

(300,)

In [77]:
print( type(X_train))
print(type(y_train))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


####Train the Model###

In [78]:
#model_training
model.fit(X_train_resampled,y_train_resampled, epochs=10, batch_size=64, validation_data=(X_test,y_test ) )

Epoch 1/10
38/38 [==============================] - 15s 305ms/step - loss: 2.2095 - accuracy: 0.7703 - val_loss: 1.2080 - val_accuracy: 0.9500
Epoch 2/10
38/38 [==============================] - 10s 252ms/step - loss: 0.7042 - accuracy: 0.9531 - val_loss: 0.4796 - val_accuracy: 0.9567
Epoch 3/10
38/38 [==============================] - 11s 296ms/step - loss: 0.2915 - accuracy: 0.9941 - val_loss: 0.2881 - val_accuracy: 0.9633
Epoch 4/10
38/38 [==============================] - 12s 304ms/step - loss: 0.1640 - accuracy: 0.9983 - val_loss: 0.1969 - val_accuracy: 0.9767
Epoch 5/10
38/38 [==============================] - 11s 293ms/step - loss: 0.1148 - accuracy: 0.9992 - val_loss: 0.1722 - val_accuracy: 0.9767
Epoch 6/10
38/38 [==============================] - 9s 243ms/step - loss: 0.0887 - accuracy: 0.9987 - val_loss: 0.2791 - val_accuracy: 0.9300
Epoch 7/10
38/38 [==============================] - 11s 297ms/step - loss: 0.0948 - accuracy: 0.9979 - val_loss: 0.1157 - val_accuracy: 0.9833


###Model Predicaton###

In [79]:
model_pred = model.predict(X_test)
predictions_test = np.round(model_pred).flatten().astype(int)

10/10 [==============================] - 0s 32ms/step


###Evaluate the model###

In [80]:
from sklearn.metrics import accuracy_score, classification_report

In [81]:
print(type(y_test), type(predictions_test))

<class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [82]:
accuracy_test = accuracy_score(y_test, predictions_test)

In [83]:
accuracy_test

0.9833333333333333

In [84]:
print(classification_report(y_test, predictions_test))

              precision    recall  f1-score   support

           0       0.25      0.33      0.29         3
           1       0.99      0.99      0.99       297

    accuracy                           0.98       300
   macro avg       0.62      0.66      0.64       300
weighted avg       0.99      0.98      0.98       300



In [85]:
print(f'Test Accuracy: {accuracy_test}')

Test Accuracy: 0.9833333333333333


In [86]:
from sklearn.metrics import confusion_matrix

In [87]:
confusion_matrix(y_test, predictions_test)

array([[  1,   2],
       [  3, 294]])

In [88]:
predictions_test.shape


(300,)